# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [3]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [4]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [5]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [6]:
split_documents[-1]

"ant markets all to themselves, with no\ncompetition anywhere in sight, because everyone knew the\nworld had come to an end, the better.\nI will accept your applause and gratitude in the form of immediate compliance.\nOK, you're right, it IS a bubble (October 2009) 189\nThank you.\n190 The Pmarca Blog Archives"

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [7]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

<a id="section_answer1"></a>

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!


#### ➡️ Answer #1:
1. Yes. 1536 is the maximum dimension for this model. When creating an embedding, we can pass the 'dimension' parameter in the API call. 
2. The short answer: OpenAI does this by truncating the embedding vector. 
The long answer: OpenAI uses Matryoshka Representation Learning (MRL) for representing the information in its embeddings. The core idea with MRL is that the first parts/dimensions of the embedding vector for "independently useful" information. That latter dimenssions (if available/allowed), add more meaning, nuance, and finer representation to the information presented in the ealier dimensions. This allows truncating the latter dimensions and still retain the core information that the whole/default size (1536) vector would have represented.  

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [8]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

#### ➡️ Answer #2:
There are multiple reasons why an async approach to generate embeddings (as used in this notebook) are better:
1. Performance: With async, we can make multiple API calls in parallel, instead of sequentially and having to wait for the responses of the calls.
1. Scalability: By leveraging async tools, like in this notebook, asyncio's gather(), we can process multiple embedding requests simultaneously, but still allow other CPU bound tasks to continue when waiting for the API response.
1. An application that has a user interacting with it in real-time can remain more responsive when using an async based approach because it doesn't have to block the processing when waiting for the API response. 
1. Many APIs, including OpenAI's in our case, allow batching, which is much cheaper than making the same number of calls individually. Because we are using an asynchronous approach, our code doesn't block when waiting for the batched response, which - typically - will has a bigger response time. [In the specific example used in our code, the batch size of 1024 is bigger than the 373 embeddings we are trying to create. Nonetheless, those many embeddings are still being sent to OpenAI in a single batch, and not individually.]

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [9]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

#### ➡️ Answer #3:
When a model (in this case OpenAI's model) is generating text, it doesn't always select the single highest probability token every time for the next token it might generate. Instead, it creates a porbability distribution over all possible next tokens, and then samples (randomly selects) from that. This is necessary for creativity, else for many given inputs, the model's "next token" will always be the same regardless of the context in which it was being used. 

To bring some control over this randomness, we can use some of the following options. That said, none of these can gaurantee a deterministic response. 

1. Set temperature = 0 and top_p = 1 to minimize randomness.
    - temperature controls the randomness of the sampling (the random selection the model makes from the probability distribution of the next set of possible tokens). Setting this to 0 picks the SINGLE most likely token at each step of token generation.
    - top_p (nucleus sampleing) by itself/alone does NOT control randomness but in conjunction with temperature = 0, can help narrow the selection to the most likely token. 
        - By itself, top_p decides how much "tail" of the probability distribution we want the model to keep when it selects the next token.  
        - For example, if top_p = 0.9, the model only considers tokens whose cumulative probability adds up to 0.9, ignoring the less-likely tokens in the remaining 10%.
        - If we set top_p < 1, the model might ignore some tokens altogether, which can sometimes produce surprising or truncated outputs if the model’s top token is outside that top-p cutoff.
    - When we set temperature to 0 and top_p = 1, we force the model to select the highest-probability token from the distribution without artificially exclusing any tokens from the distribution. This results in a consistent "greedy" token selection, and hence, the least possible randomness. 
1. Keep other parameters neutral:
    - As with top_p, by themselves parameters like frequency_penalty and presence_penalty don't directly get us to a consistent output, but because they do influence the model's token selection, it is recommended that they be set to 0. 
1. For a sustained reproducible output over weeks/months, we should use a versioned model:
    - We should specify a static/dated model version instead of the latest/rolling version. That is, gpt-3.5-turbo-0301 instead of gpt-3.5-turbo or gpt-4o-2024-11-20 instead of gpt-4o.
1. Ensure prompts, roles, and context used are the same:
    - If the prompt and context we give to the model is different than earlier, then even if the parameters are identical, the model's response will likely be different than earlier.
    - For reproducing an output consistent with a previous interaction with a model, we should ensure our prompts and conversation history are 'exactly' the same.
        - Entire conversation history, including whitepace and punctuation, should be the same.
        - If streaming or chunking a conversation, each chunk should be fed to the model in the same manner, size, etc. as before.
    

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [10]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [11]:
print(response)

The best way to write a loop in Python largely depends on your specific use case, but I can share some general guidelines and examples for common types of loops: `for` loops and `while` loops.

### Using a `for` loop

For iterating over a sequence (like a list or a range of numbers), `for` loops are often the best choice for their readability and simplicity. Here's an example of a `for` loop that prints each item in a list:

```python
fruits = ['apple', 'banana', 'cherry']

for fruit in fruits:
    print(fruit)
```

You can also use `range()` to loop through a sequence of numbers:

```python
for i in range(5):
    print(i)
```

### Using a `while` loop

A `while` loop is useful when you want to repeat an action until a certain condition is met. Here's an example that counts down from 5:

```python
count = 5
while count > 0:
    print(count)
    count -= 1
```

### Recommendations for Writing Loops

1. **Keep it Simple**: Strive for clarity. Avoid complex nested loops if possible; they 

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [16]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

#### ➡️ Answer #4:
Here are some ways with which we can guide a model to give more thoughtful and detaield responses:

1. Provide context: Include relevant background information or specify the purpose of the query.
1. Ask open-ended questions: This encourages the model to explore complex topics and generate more thoughtful and detailed responses.
1. Chain-of-Thought prompting: Ask the model to break down its reasoning into intermediate, sequential steps before attempting to reach a final answer. This can be done by a prompt like "Think step-by-step" or by a more task specific "To resolve A, first think about B, and then think about C". 
1. Give direct instructions: Clearly outline what the model should do. These could specifystep-by-step instructions or the desired output format that forces the model to give a more thoughtful and detailed response.
1. If the application/interface allows, ask the model to ask clarifying questions before attempting to generate the final answer.This causes the model to consider different aspects before answering. 
1. Few-Shot approach: Give the model a few examples of the kind of thoughtful, detailed responses you expect.
1. Fine-tune the model: Train the model on domain specific data with the training data guiding the model into responding with more thoughtful and detailed responses.


In [19]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [20]:
retrieval_augmented_qa_pipeline.run_pipeline("What is the 'Michael Eisner Memorial Weak Executive Problem'?")

{'response': "The 'Michael Eisner Memorial Weak Executive Problem' refers to a situation where a CEO or founder, who has a strong background in a particular area (such as product management, sales, or marketing), inadvertently hires a weak executive in that same area. This often happens because they may not want to fully let go of their expertise and instead opt to hire someone less competent, allowing them to maintain control and continue to be seen as the knowledgeable authority. The term is named after Michael Eisner, the former CEO of Disney, who, despite his success as a TV network executive, faced challenges with ABC after its acquisition, highlighting the pitfalls of hiring less capable individuals for key roles.",
 'context': [('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\np

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

--- 

### UPDATE 1: 
A dot-product similarity has been implemented in aimakerspace.vectordatabase.py and it replaces the original cosine similarity function.
That is the distance metric function that is now used in the pipeline in this notebook. 

---


### UPDATE 2: 
Implement a PDF loader.
A PDF file loader has been implemented in aimakerspace.pdf_utils.
Similar to the text file loader, it loads PDF files from 'data/pdf/' folder.
Currently that folder has some AWS Whitepapers. 

In [22]:
from aimakerspace.pdf_utils import PDFFileLoader

pdf_loader = PDFFileLoader("data/pdf")
documents_pdf = pdf_loader.load_documents()
len(documents_pdf)


2

The above should show how many PDFs were read.

Below:
1. we split the PDFs that were read into chunks using the character text splitter.
1. display the number of chunks.
1. show the contents of the second-last chunk.

In [27]:
splitter = CharacterTextSplitter()
chunks_pdf_text = splitter.split_texts(pdf_loader.documents)
print(f'Number of chunks: {len(chunks_pdf_text)}')
print(f'\nLast chunk contents:\n{chunks_pdf_text[-2]}\n')

Number of chunks: 2817

Last chunk contents:
iew your testing results and make necessary improvements.
Process and culture 993
AWS Well-Architected Framework Framework
Resources
Related documents:
•AWS Device Farm device list
•Viewing the CloudWatch RUM dashboard
Related videos:
•AWS re:Invent 2023 - Improve your mobile and web app quality using AWS Device Farm
•AWS re:Invent 2021 - Optimize applications through end user insights with Amazon CloudWatch 
RUM
Related examples:
•AWS Device Farm Sample App for Android
•AWS Device Farm Sample App for iOS
•Appium Web tests for AWS Device Farm
Process and culture 994
AWS Well-Architected Framework Framework
Notices
Customers are responsible for making their own independent assessment of the information in 
this document. This document: (a) is for informational purposes only, (b) represents current AWS 
product oﬀerings and practices, which are subject to change without notice, and (c) does not create 
any commitments or assurances from AWS an

We now create a vector database using the chunks created from the PDF file, along with the RAG pipeline.

In [29]:
vector_db_pdf = VectorDatabase()
vector_db_pdf = asyncio.run(vector_db_pdf.abuild_from_list(chunks_pdf_text))

retrieval_augmented_qa_pipeline_pdf = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db_pdf,
    llm=chat_openai
)


We now run the RAG pipeline with a question we want answered using the contents of the PDF files.

In [ ]:
question_for_pdf_content: str = "How should ML endpoints be exposed to allow model updates while serving requests?"
llm_return = retrieval_augmented_qa_pipeline_pdf.run_pipeline(question_for_pdf_content)

We now display the response to the question we were asked:

In [33]:
response = llm_return.get("response", "")
context = llm_return.get("context", "")

print(f'Response: \n{response}')
#print(f'Context: \n{context}')

Response: 
ML endpoints should be exposed through APIs to abstract changes from model consuming applications. This design ensures that changes to an ML model can be introduced with minimal or no interruption to existing workload capabilities, effectively minimizing the impact on downstream applications. It is also recommended to document the API in a central repository or documentation site for clarity and consistency.
